# DuckDB / Ibis / deltalake

In [1]:
import os

import pandas as pd
from deltalake import DeltaTable
from deltalake.writer import write_deltalake

# Create a Delta table

In [2]:
df = pd.DataFrame({"x": ["a", "b", "c", "a"], "y": [1, 2, 3, 4]})

In [3]:
df

,x,y
0,a,1
1,b,2
2,c,3
3,a,4


In [4]:
os.makedirs("tmp/some_delta_lake", exist_ok=True)

In [5]:
write_deltalake("tmp/some_delta_lake", df)

## Append to the Delta table

In [6]:
df = pd.DataFrame({"x": ["l", "m"], "y": [20, 30]})

In [7]:
df

,x,y
0,l,20
1,m,30


In [8]:
write_deltalake("tmp/some_delta_lake", df, mode="append")

## Create an Ibis table from the Delta table

In [9]:
import ibis

In [14]:
con = ibis.connect("duckdb://")

In [17]:
con.read_in_memory(DeltaTable("tmp/some_delta_lake").to_pyarrow_dataset(), table_name="something")

┏━━━━━━━━┳━━━━━━━┓
┃ x      ┃ y     ┃
┡━━━━━━━━╇━━━━━━━┩
│ string │ int64 │
├────────┼───────┤
│ a      │     1 │
│ b      │     2 │
│ c      │     3 │
│ a      │     4 │
│ l      │    20 │
│ m      │    30 │
└────────┴───────┘

In [18]:
con.table("something").head(2).execute()

,x,y
0,a,1
1,b,2


## Time travel back to version 0 of the data

In [20]:
con.read_in_memory(DeltaTable("tmp/some_delta_lake", version=0).to_pyarrow_dataset(), table_name="somethingv0")

┏━━━━━━━━┳━━━━━━━┓
┃ x      ┃ y     ┃
┡━━━━━━━━╇━━━━━━━┩
│ string │ int64 │
├────────┼───────┤
│ a      │     1 │
│ b      │     2 │
│ c      │     3 │
│ a      │     4 │
└────────┴───────┘

In [23]:
somethingv0 = con.table("somethingv0")

In [24]:
somethingv0.filter(somethingv0.y >= 4).execute()

,x,y
0,a,4
